# Exploring conformational space of "m13+1H2O"

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Fri Apr 28 15:11:49 2017


In [2]:
# Functions used in this notebook:

def grep_energies_from_sdf_outputs(files):
    energies = {}
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies[os.path.splitext(os.path.basename(inp))[0]] = float(lines[i+1])
    return energies

def find_duplicates(rms_sorted, energy, rms_thresh):
    i = 0
    to_be_deleted = []
    while i < len(rms_sorted):
        j = i + 1
        while j < len(rms_sorted):
            if rms_sorted[i][0] in to_be_deleted:
                i = i + 1
                j = j + 1
            elif rms_sorted[j][0] in to_be_deleted:
                j = j + 1
            else:
                rms1 = rms_sorted[i][1]
                rms2 = rms_sorted[j][1]
                if (rms2 - rms1) < rms_thresh:
                    if energy[rms_sorted[i][0]] < energy[rms_sorted[j][0]]:
                        to_be_deleted.append(rms_sorted[j][0])
                    else:
                        to_be_deleted.append(rms_sorted[i][0])
                else:
                    break
        i = i + 1
    if to_be_deleted:
        print("Conformers which will be deleted:")    
        print(to_be_deleted)
    return to_be_deleted

In [3]:
# useful for later:
# "core" is a part of a molecule, which we wish to be the "most-aligned" among multiple conformers
smiles      = 'N1C(=O)c2cc(C(=O)NCCCCCNC(=O)c3cc(C(=O)NCCCCC1)cc(c3)C(C)(C)C)cc(c2)C(C)(C)C'
core_smiles = 'c1ccccc1'

m13 = Chem.AddHs(Chem.MolFromSmiles(smiles))
core_m13 = m13.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))

templ_m13 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/m13_crystal.sdf')
m13_crystal = templ_m13[0]

## m13 + H2O

m13 + H2O structures were generated from the crystal geometry of isolated m13 macrocycle, to which we have added :

* 1 H2O molecule "inside" the macrocycle ("m13_h2o_in", left fig. below)

* 1 H2O molecule "outside" the macrocycle ("m13_h2o_out", right fig. below)

manually in Avogadro software; then pre-optimized with MM implemented in Avogadro.

On figures below we present already pre-optimized "m13 + h2o" complexes, which serve as starting geometries for exploration of the conformational space.

In [4]:
%%html
<table>
  <tr>
    <td id="m13_h2o_in" ></td>
    <td id="m13_h2o_out" ></td>
  <tr>
    <td> m13_h2o_in </td>
    <td> m13_h2o_out  </td>  
  </tr>
</table>

,
m13_h2o_in,m13_h2o_out


In [5]:
m_in = open('/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/m13_h2o_in.xyz', 'r').read()
p_in = py3Dmol.view(width=300,height=300)
p_in.addModel(m_in,'xyz')
p_in.setStyle({'stick':{'radius':'0.15'}})
p_in.setBackgroundColor('0xeeeeee')
p_in.zoomTo()
p_in.insert('m13_h2o_in')

In [6]:
m_out = open('/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/m13_h2o_out.xyz', 'r').read()
p_out = py3Dmol.view(width=300,height=300)
p_out.addModel(m_out,'xyz')
p_out.setStyle({'stick':{'radius':'0.15'}})
p_out.setBackgroundColor('0xeeeeee')
p_out.zoomTo()
p_out.insert('m13_h2o_out')

### Conformers generated with the Balloon software:

In both cases the Balloon software was asked to generate 100 conformers using the genertic algorithm with default settings (only "maxPostprocessIter" increased to 150 and "nGenerations" to 300).

Below we will present aligned structures of macrocycle m13 conformers in "m13 + H2O" (without showing H2O molecules).

In [7]:
inps_m13_h2o_in_b  = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/balloon/results_starting_from_m13_h2o_in/*.sdf')
inps_m13_h2o_out_b = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/balloon/results_starting_from_m13_h2o_out/*.sdf')

In [8]:
e_m13_h2o_in_b  = grep_energies_from_sdf_outputs(inps_m13_h2o_in_b)
e_m13_h2o_out_b = grep_energies_from_sdf_outputs(inps_m13_h2o_out_b)

In [9]:
%%html
<table>
  <tr>
    <td id="m13_h2o_in_b" ></td>
    <td id="m13_h2o_out_b"  ></td>
  <tr>
    <td> m13_h2o_in_b </td>
    <td> m13_h2o_out_b  </td>  
  </tr>
</table>

,
m13_h2o_in_b,m13_h2o_out_b


In [10]:
# write conformers to dictionaries
allmol_m13_h2o_in_b  = {}
allmol_m13_h2o_out_b = {}
suppl_m13_h2o_in_b  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/balloon/m13_h2o_in_result.sdf')
suppl_m13_h2o_out_b = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/balloon/m13_h2o_out_result.sdf')

for i, mol in enumerate(suppl_m13_h2o_in_b):
    name = "m13_h2o_in_b_" + str(i)
    allmol_m13_h2o_in_b[name] = mol
for i, mol in enumerate(suppl_m13_h2o_out_b):
    name = "m13_h2o_out_b_" + str(i)
    allmol_m13_h2o_out_b[name] = mol 

In [11]:
# align:
for key, mol in allmol_m13_h2o_in_b.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
for key, mol in allmol_m13_h2o_out_b.items():   
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))    

In [12]:
# view:
p1_b_handles=[]

p1_b_in = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_h2o_in_b.items():
    mb = Chem.MolToMolBlock(mol)
    p1_b_in.addModel(mb,'sdf')
p1_b_in.setStyle({'stick':{'radius':'0.15'}})
p1_b_in.setBackgroundColor('0xeeeeee')
p1_b_in.zoomTo()    
p1_b_handles.append(p1_b_in)

p1_b_out = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_h2o_out_b.items():
    mb = Chem.MolToMolBlock(mol)
    p1_b_out.addModel(mb,'sdf')
p1_b_out.setStyle({'stick':{'radius':'0.15'}})
p1_b_out.setBackgroundColor('0xeeeeee')
p1_b_out.zoomTo()    
p1_b_handles.append(p1_b_out)

In [13]:
p1_b_handles[0].insert('m13_h2o_in_b')

In [14]:
p1_b_handles[1].insert('m13_h2o_out_b')

In [16]:
allmol_m13_h2o_b = {}
allmol_m13_h2o_b.update(allmol_m13_h2o_in_b)
allmol_m13_h2o_b.update(allmol_m13_h2o_out_b)

with open("/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/list_selected_conformers_from_ballon", "w") as f:
    for key, mol in allmol_m13_h2o_b.items():
        f.write(key+"\n")

### pre-screening

Some of the generated conformers are very much alike. To remove potential duplicates which were not "caught" by the Balloon program, we can compare the energies (preoptimized with MM) and the RMSD calculated against a reference structure (here: the crystal structure of m13). It does not matter against which structure we are aligning the conformers, since we are interested in relative RMS between them.

First let's print the energies and RMS values:

#### "in"

In [16]:
allmol_m13_h2o_b = {}
allmol_m13_h2o_b.update(allmol_m13_h2o_in_b)

energy_m13_h2o_b = {}
energy_m13_h2o_b.update(e_m13_h2o_in_b)

rms_m13_h2o_b = {}
for key, mol in allmol_m13_h2o_b.items():
    rms_m13_h2o_b[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m13_crystal))

Then we can introduce some thresholds, for instance:

* if two conformers differ by less than 0.01 in RMS (measured against the reference structure), then select the one with the lower energy

In [17]:
rms_sorted = sorted(rms_m13_h2o_b.items(), key=lambda x: x[1])
rms_thresh = 0.05

print("List sorted by RMS:")
for i, t in enumerate(rms_sorted):
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(rms_sorted[i][0], energy_m13_h2o_b[rms_sorted[i][0]], rms_sorted[i][1]))

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m13_h2o_b, rms_thresh)

for mol in to_be_deleted:
    del allmol_m13_h2o_b[mol]
    del energy_m13_h2o_b[mol]
    del rms_m13_h2o_b[mol]  

List sorted by RMS:
name = m13_h2o_in_b_15, E = 57.582827, RMS = 0.316532
name = m13_h2o_in_b_9, E = 53.096084, RMS = 0.343899
name = m13_h2o_in_b_4, E = 52.014900, RMS = 0.427144
name = m13_h2o_in_b_10, E = 53.446733, RMS = 0.463845
name = m13_h2o_in_b_6, E = 52.215628, RMS = 0.494019
name = m13_h2o_in_b_16, E = 58.387284, RMS = 0.520676
name = m13_h2o_in_b_14, E = 55.832805, RMS = 0.626317
name = m13_h2o_in_b_0, E = 51.092666, RMS = 0.665046
name = m13_h2o_in_b_2, E = 51.828845, RMS = 0.670723
name = m13_h2o_in_b_3, E = 51.860661, RMS = 0.687435
name = m13_h2o_in_b_13, E = 55.299894, RMS = 0.710594
name = m13_h2o_in_b_7, E = 52.238166, RMS = 0.759545
name = m13_h2o_in_b_1, E = 51.472907, RMS = 0.801903
name = m13_h2o_in_b_8, E = 52.646483, RMS = 0.872269
name = m13_h2o_in_b_5, E = 52.099432, RMS = 1.130466
name = m13_h2o_in_b_11, E = 53.475374, RMS = 1.258449
name = m13_h2o_in_b_12, E = 53.765350, RMS = 1.275817
Conformers which will be deleted:
['m13_h2o_in_b_15', 'm13_h2o_in_b_10',

Below we will align the selected conformers:

In [18]:
for key, mol in allmol_m13_h2o_b.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
    
p_b = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_h2o_b.items():
    mb = Chem.MolToMolBlock(mol)
    p_b.addModel(mb,'sdf')
p_b.setStyle({'stick':{'radius':'0.15'}})
p_b.setBackgroundColor('0xeeeeee')
p_b.zoomTo()
p_b.show()

In [19]:
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/list_selected_conformers_from_ballon_in", "w") as f:
    for key, mol in allmol_m13_h2o_b.items():
        f.write(key+"\n")    

#### "out"

In [20]:
allmol_m13_h2o_b = {}
allmol_m13_h2o_b.update(allmol_m13_h2o_out_b)

energy_m13_h2o_b = {}
energy_m13_h2o_b.update(e_m13_h2o_out_b)

rms_m13_h2o_b = {}
for key, mol in allmol_m13_h2o_b.items():
    rms_m13_h2o_b[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m13_crystal))

In [21]:
rms_sorted = sorted(rms_m13_h2o_b.items(), key=lambda x: x[1])
rms_thresh = 0.05

print("List sorted by RMS:")
for i, t in enumerate(rms_sorted):
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(rms_sorted[i][0], energy_m13_h2o_b[rms_sorted[i][0]], rms_sorted[i][1]))

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m13_h2o_b, rms_thresh)

for mol in to_be_deleted:
    del allmol_m13_h2o_b[mol]
    del energy_m13_h2o_b[mol]
    del rms_m13_h2o_b[mol]  

List sorted by RMS:
name = m13_h2o_out_b_16, E = 56.240346, RMS = 0.198288
name = m13_h2o_out_b_14, E = 54.789314, RMS = 0.223653
name = m13_h2o_out_b_12, E = 54.058498, RMS = 0.331301
name = m13_h2o_out_b_9, E = 52.959594, RMS = 0.441097
name = m13_h2o_out_b_15, E = 56.119637, RMS = 0.542196
name = m13_h2o_out_b_4, E = 52.087903, RMS = 0.564221
name = m13_h2o_out_b_0, E = 51.210724, RMS = 0.596499
name = m13_h2o_out_b_13, E = 54.225384, RMS = 0.597136
name = m13_h2o_out_b_2, E = 51.919702, RMS = 0.613838
name = m13_h2o_out_b_1, E = 51.269769, RMS = 0.636244
name = m13_h2o_out_b_18, E = 60.737330, RMS = 0.716464
name = m13_h2o_out_b_8, E = 52.587915, RMS = 0.728755
name = m13_h2o_out_b_3, E = 52.050788, RMS = 0.734629
name = m13_h2o_out_b_19, E = 60.766323, RMS = 0.827590
name = m13_h2o_out_b_17, E = 56.683678, RMS = 0.862052
name = m13_h2o_out_b_6, E = 52.351251, RMS = 0.988618
name = m13_h2o_out_b_5, E = 52.164343, RMS = 0.988768
name = m13_h2o_out_b_7, E = 52.464592, RMS = 1.197651


In [22]:
for key, mol in allmol_m13_h2o_b.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
    
p_b = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_h2o_b.items():
    mb = Chem.MolToMolBlock(mol)
    p_b.addModel(mb,'sdf')
p_b.setStyle({'stick':{'radius':'0.15'}})
p_b.setBackgroundColor('0xeeeeee')
p_b.zoomTo()
p_b.show()

In [23]:
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/list_selected_conformers_from_ballon_out", "w") as f:
    for key, mol in allmol_m13_h2o_b.items():
        f.write(key+"\n")  

## Conformers generated with the RDKit software:

In [17]:
inps_m13_h2o_in_rdkit  = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/rdkit/results_starting_from_m13_h2o_in/*.sdf')
inps_m13_h2o_out_rdkit = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/rdkit/results_starting_from_m13_h2o_out/*.sdf')

In [18]:
e_m13_h2o_in_rdkit = grep_energies_from_sdf_outputs(inps_m13_h2o_in_rdkit)
e_m13_h2o_out_rdkit = grep_energies_from_sdf_outputs(inps_m13_h2o_out_rdkit)

In [19]:
# write conformers to dictionaries
allmol_m13_h2o_in_rdkit = {}
suppl_m13_h2o_in_rdkit  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/rdkit/m13_h2o_in_result.sdf')
allmol_m13_h2o_out_rdkit = {}
suppl_m13_h2o_out_rdkit  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/rdkit/m13_h2o_out_result.sdf')

for i, mol in enumerate(suppl_m13_h2o_in_rdkit):
    name = "m13_h2o_in_rdkit_" + str(i)
    allmol_m13_h2o_in_rdkit[name] = mol  
for i, mol in enumerate(suppl_m13_h2o_out_rdkit):
    name = "m13_h2o_out_rdkit_" + str(i)
    allmol_m13_h2o_out_rdkit[name] = mol     

In [20]:
# align:
for key, mol in allmol_m13_h2o_in_rdkit.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
for key, mol in allmol_m13_h2o_out_rdkit.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles)) 
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))

In [21]:
%%html
<table>
  <tr>
    <td id="m13_h2o_in_rdkit" ></td>
    <td id="m13_h2o_out_rdkit"  ></td>
  <tr>
    <td> m13_h2o_in_rdkit </td>
    <td> m13_h2o_out_rdkit  </td>  
  </tr>
</table>

,
m13_h2o_in_rdkit,m13_h2o_out_rdkit


In [22]:
# view:
p1_h2o_rdkit_handles=[]

p1_h2o_in_rdkit = py3Dmol.view(width=400,height=400)
p1_h2o_in_rdkit.removeAllModels()
for key, mol in allmol_m13_h2o_in_rdkit.items(): 
    mb = Chem.MolToMolBlock(mol)
    p1_h2o_in_rdkit.addModel(mb,'sdf')    
p1_h2o_in_rdkit.setStyle({'stick':{'radius':'0.15'}})
p1_h2o_in_rdkit.setBackgroundColor('0xeeeeee')
p1_h2o_in_rdkit.zoomTo()
p1_h2o_rdkit_handles.append(p1_h2o_in_rdkit)

p1_h2o_out_rdkit = py3Dmol.view(width=400,height=400)
p1_h2o_out_rdkit.removeAllModels()
for key, mol in allmol_m13_h2o_out_rdkit.items(): 
    mb = Chem.MolToMolBlock(mol)
    p1_h2o_out_rdkit.addModel(mb,'sdf')    
p1_h2o_out_rdkit.setStyle({'stick':{'radius':'0.15'}})
p1_h2o_out_rdkit.setBackgroundColor('0xeeeeee')
p1_h2o_out_rdkit.zoomTo()
p1_h2o_rdkit_handles.append(p1_h2o_out_rdkit)

In [23]:
p1_h2o_rdkit_handles[0].insert('m13_h2o_in_rdkit')

In [24]:
p1_h2o_rdkit_handles[1].insert('m13_h2o_out_rdkit')

### pre-screening

In [25]:
allmol_m13_h2o_rdkit = {}
allmol_m13_h2o_rdkit.update(allmol_m13_h2o_in_rdkit)
allmol_m13_h2o_rdkit.update(allmol_m13_h2o_out_rdkit)

energy_m13_h2o_rdkit = {}
energy_m13_h2o_rdkit.update(e_m13_h2o_in_rdkit)
energy_m13_h2o_rdkit.update(e_m13_h2o_out_rdkit)

rms_m13_h2o_rdkit = {}
for key, mol in allmol_m13_h2o_rdkit.items():
    rms_m13_h2o_rdkit[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m13_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m13_rdkit[key], rms_m13_rdkit[key]))

In [26]:
rms_sorted = sorted(rms_m13_h2o_rdkit.items(), key=lambda x: x[1])
rms_thresh = 0.05


# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m13_h2o_rdkit, rms_thresh)

for mol in to_be_deleted:
    del allmol_m13_h2o_rdkit[mol]
    del energy_m13_h2o_rdkit[mol]
    del rms_m13_h2o_rdkit[mol]  

Conformers which will be deleted:
['m13_h2o_out_rdkit_89', 'm13_h2o_out_rdkit_4', 'm13_h2o_in_rdkit_22', 'm13_h2o_out_rdkit_22', 'm13_h2o_in_rdkit_61', 'm13_h2o_out_rdkit_77', 'm13_h2o_in_rdkit_77', 'm13_h2o_in_rdkit_62', 'm13_h2o_out_rdkit_62', 'm13_h2o_in_rdkit_6', 'm13_h2o_in_rdkit_13', 'm13_h2o_out_rdkit_13', 'm13_h2o_out_rdkit_6', 'm13_h2o_in_rdkit_44', 'm13_h2o_out_rdkit_44', 'm13_h2o_out_rdkit_7', 'm13_h2o_in_rdkit_65', 'm13_h2o_in_rdkit_28', 'm13_h2o_out_rdkit_28', 'm13_h2o_out_rdkit_58', 'm13_h2o_in_rdkit_58', 'm13_h2o_out_rdkit_10', 'm13_h2o_out_rdkit_63', 'm13_h2o_in_rdkit_63', 'm13_h2o_out_rdkit_36', 'm13_h2o_in_rdkit_36', 'm13_h2o_out_rdkit_17', 'm13_h2o_in_rdkit_17', 'm13_h2o_out_rdkit_23', 'm13_h2o_in_rdkit_98', 'm13_h2o_out_rdkit_98', 'm13_h2o_out_rdkit_19', 'm13_h2o_in_rdkit_19', 'm13_h2o_in_rdkit_23', 'm13_h2o_in_rdkit_79', 'm13_h2o_out_rdkit_11', 'm13_h2o_in_rdkit_11', 'm13_h2o_out_rdkit_93', 'm13_h2o_in_rdkit_93', 'm13_h2o_out_rdkit_1', 'm13_h2o_in_rdkit_1', 'm13_h2

Below we will align the selected conformers:

In [27]:
for key, mol in allmol_m13_h2o_rdkit.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
    
p_r = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_h2o_rdkit.items():
    mb = Chem.MolToMolBlock(mol)
    p_r.addModel(mb,'sdf')
p_r.setStyle({'stick':{'radius':'0.15'}})
p_r.setBackgroundColor('0xeeeeee')
p_r.zoomTo()
p_r.show()

In [28]:
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/list_selected_conformers_from_rdkit", "w") as f:
    for key, mol in allmol_m13_h2o_rdkit.items():
        f.write(key+"\n")  

## Summary

Now let's generate a list of all conformers (from all programs used, as presented above). We can further pre-screen all the structures and remove potential duplicates. Here we can also use more crude RMSD threshold.

In [27]:
allmol_m13_h2o = {}
allmol_m13_h2o.update(allmol_m13_h2o_b)
allmol_m13_h2o.update(allmol_m13_h2o_rdkit)

energy_m13_h2o = {}
energy_m13_h2o.update(energy_m13_h2o_b)
energy_m13_h2o.update(energy_m13_h2o_rdkit)

rms_m13_h2o = {}
for key, mol in allmol_m13_h2o.items():
    rms_m13_h2o[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m13_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m13[key], rms_m13[key]))

NameError: name 'allmol_m13_h2o_b' is not defined

In [36]:
rms_sorted = sorted(rms_m13_h2o.items(), key=lambda x: x[1])
rms_thresh = 0.1

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m13_h2o, rms_thresh)

for mol in to_be_deleted:
    del allmol_m13_h2o[mol]
    del energy_m13_h2o[mol]
    del rms_m13_h2o[mol]  

Conformers which will be deleted:
['m13_h2o_in_rdkit_19', 'm13_h2o_in_rdkit_34', 'm13_h2o_out_b_10', 'm13_h2o_out_rdkit_11', 'm13_h2o_in_rdkit_14', 'm13_h2o_in_rdkit_2', 'm13_h2o_out_rdkit_32', 'm13_h2o_in_rdkit_0']


Finally we can align all conformers which will further be used as starting points in DFT geometry optimizations:

In [37]:
print(len(allmol_m13_h2o))

for key, mol in allmol_m13_h2o.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
    
p = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_h2o.items():
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

13


Write the selected conformers' names to the list "list_selected_conformers_from_balloon_rdkit". It will be used to generate Gaussian inputs:

In [38]:
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/list_selected_conformers_from_ballon_rdkit", "w") as f:
    for key, mol in allmol_m13_h2o.items():
        f.write(key+"\n")

energy_sorted = sorted(energy_m13_h2o.items(), key=lambda x: x[1])
print(energy_sorted)
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m13-1h2o/detailed_list_selected_conformers_from_ballon_rdkit", "w") as f:
    for pair in energy_sorted:
        f.write("{0:30}   {1}\n".format(pair[0], pair[1]))          

[('m13_h2o_out_b_0', 51.21072384846502), ('m13_h2o_out_b_3', 52.05078819235895), ('m13_h2o_out_b_5', 52.1643431238253), ('m13_h2o_out_b_7', 52.46459240004303), ('m13_h2o_out_b_9', 52.959593601332905), ('m13_h2o_out_rdkit_41', 53.73), ('m13_h2o_out_b_12', 54.05849757183306), ('m13_h2o_out_b_14', 54.789313762111476), ('m13_h2o_out_rdkit_4', 55.09), ('m13_h2o_in_rdkit_11', 55.42), ('m13_h2o_out_rdkit_1', 55.66), ('m13_h2o_in_rdkit_26', 56.06), ('m13_h2o_out_b_17', 56.68367774518494)]
